### Cal-CRAI metric calculation for flood insurance policies
* Enrollment in national flood insurance program -- community preparedness
* num. of NFIP participants -- personal preparedness

In [1]:
import pandas as pd
import os
import sys
import math

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))

from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/governance/community_preparedness/fema/nfip_community_status/fema_community_status_subset.csv'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'fema_community_status_subset.csv'


In [3]:
community_flood_insurance_data = pd.read_csv('fema_community_status_subset.csv')
print(len(community_flood_insurance_data))
community_flood_insurance_data.head(5)
# os.remove('fema_community_status_subset.csv')

533


,communityIdNumber,communityName,county,state,initialFloodHazardBoundaryMap,initialFloodInsuranceRateMap,currentlyEffectiveMapDate,regularEmergencyProgramDate,tribal,participatingInNFIP,originalEntryDate,classRatingEffectiveDate,classRating,sfhaDiscount,nonSfhaDiscount,lastRefresh
0,60262,SACRAMENTO COUNTY *,SACRAMENTO COUNTY,CA,1975-01-10T00:00:00.000Z,1979-03-15T00:00:00.000Z,02/22/24(>),03/15/79,0,1,1992-10-01T00:00:00.000Z,2017-05-01T00:00:00.000Z,2.0,40%,10%,2023-10-16T01:01:20.073Z
1,60263,"FOLSOM, CITY OF",SACRAMENTO COUNTY,CA,1976-12-03T00:00:00.000Z,1982-01-06T00:00:00.000Z,02/22/24(>),01/06/82,0,1,NaN,NaN,NaN,NaN,NaN,2023-10-16T01:01:20.079Z
2,60264,"GALT, CITY OF",SACRAMENTO COUNTY,CA,1974-05-17T00:00:00.000Z,1981-12-01T00:00:00.000Z,10/20/16,12/01/81,0,1,NaN,NaN,NaN,NaN,NaN,2023-10-16T01:01:20.085Z
3,60266,"SACRAMENTO, CITY OF",SACRAMENTO COUNTY,CA,1975-01-10T00:00:00.000Z,1978-09-15T00:00:00.000Z,02/22/24(>),09/15/78,0,1,1991-10-01T00:00:00.000Z,2021-10-01T00:00:00.000Z,3.0,35%,10%,2023-10-16T01:01:20.093Z
4,60195,MONTEREY COUNTY *,MONTEREY COUNTY,CA,1978-02-21T00:00:00.000Z,1984-01-30T00:00:00.000Z,11/30/23(>),01/30/84,0,1,1991-10-01T00:00:00.000Z,2020-10-01T00:00:00.000Z,5.0,25%,10%,2023-09-19T20:31:30.265Z


In [4]:
community_flood_insurance_data.columns

Index(['communityIdNumber', 'communityName', 'county', 'state',
       'initialFloodHazardBoundaryMap', 'initialFloodInsuranceRateMap',
       'currentlyEffectiveMapDate', 'regularEmergencyProgramDate', 'tribal',
       'participatingInNFIP', 'originalEntryDate', 'classRatingEffectiveDate',
       'classRating', 'sfhaDiscount', 'nonSfhaDiscount', 'lastRefresh'],
      dtype='object')

## Adjust the columns and entries within for consistency

In [5]:
community_flood_insurance_data.columns = community_flood_insurance_data.columns.str.lower()
community_flood_insurance_data = community_flood_insurance_data.applymap(lambda s: s.lower() if type(s) == str else s)
community_flood_insurance_data['county'] = community_flood_insurance_data['county'].str.replace(' county', '', case=False)

community_flood_insurance_data

,communityidnumber,communityname,county,state,initialfloodhazardboundarymap,initialfloodinsuranceratemap,currentlyeffectivemapdate,regularemergencyprogramdate,tribal,participatinginnfip,originalentrydate,classratingeffectivedate,classrating,sfhadiscount,nonsfhadiscount,lastrefresh
0,60262,sacramento county *,sacramento,ca,1975-01-10t00:00:00.000z,1979-03-15t00:00:00.000z,02/22/24(>),03/15/79,0,1,1992-10-01t00:00:00.000z,2017-05-01t00:00:00.000z,2.0,40%,10%,2023-10-16t01:01:20.073z
1,60263,"folsom, city of",sacramento,ca,1976-12-03t00:00:00.000z,1982-01-06t00:00:00.000z,02/22/24(>),01/06/82,0,1,NaN,NaN,NaN,NaN,NaN,2023-10-16t01:01:20.079z
2,60264,"galt, city of",sacramento,ca,1974-05-17t00:00:00.000z,1981-12-01t00:00:00.000z,10/20/16,12/01/81,0,1,NaN,NaN,NaN,NaN,NaN,2023-10-16t01:01:20.085z
3,60266,"sacramento, city of",sacramento,ca,1975-01-10t00:00:00.000z,1978-09-15t00:00:00.000z,02/22/24(>),09/15/78,0,1,1991-10-01t00:00:00.000z,2021-10-01t00:00:00.000z,3.0,35%,10%,2023-10-16t01:01:20.093z
4,60195,monterey county *,monterey,ca,1978-02-21t00:00:00.000z,1984-01-30t00:00:00.000z,11/30/23(>),01/30/84,0,1,1991-10-01t00:00:00.000z,2020-10-01t00:00:00.000z,5.0,25%,10%,2023-09-19t20:31:30.265z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,60056,"selma, city of",fresno,ca,1974-05-24t00:00:00.000z,2001-07-19t00:00:00.000z,02/18/09,05/24/75,0,0,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:33:43.169z
529,60249,"coachella, city of",riverside,ca,1974-05-17t00:00:00.000z,1980-09-30t00:00:00.000z,03/06/18,03/07/18(s),0,0,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:33:43.174z
530,60436,"trinidad, city of",humboldt,ca,1976-05-14t00:00:00.000z,2016-11-04t00:00:00.000z,06/21/17,05/14/77,0,0,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:33:43.181z
531,60447,"gustine, city of",merced,ca,NaN,2008-12-02t00:00:00.000z,12/02/08,12/02/09,0,0,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:33:43.188z


## Utilize the communityname column's astrisk within its entries, which indicate a total communities nfip involvement
* look at just entries with the astrisk and identify counties with zero nfip participation

In [45]:
isolated_flood_insurance_counties = community_flood_insurance_data[community_flood_insurance_data['communityname'].str.endswith('*')]
isolated_flood_insurance_counties.head()

,communityidnumber,communityname,county,state,initialfloodhazardboundarymap,initialfloodinsuranceratemap,currentlyeffectivemapdate,regularemergencyprogramdate,tribal,participatinginnfip,originalentrydate,classratingeffectivedate,classrating,sfhadiscount,nonsfhadiscount,lastrefresh
0,60262,sacramento county *,sacramento,ca,1975-01-10t00:00:00.000z,1979-03-15t00:00:00.000z,02/22/24(>),03/15/79,0,1,1992-10-01t00:00:00.000z,2017-05-01t00:00:00.000z,2.0,40%,10%,2023-10-16t01:01:20.073z
4,60195,monterey county *,monterey,ca,1978-02-21t00:00:00.000z,1984-01-30t00:00:00.000z,11/30/23(>),01/30/84,0,1,1991-10-01t00:00:00.000z,2020-10-01t00:00:00.000z,5.0,25%,10%,2023-09-19t20:31:30.265z
6,60632,alpine county*,alpine,ca,NaN,1987-11-19t00:00:00.000z,11/16/23(>),04/19/89,0,1,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:31:33.591z
7,60001,alameda county*,alameda,ca,1974-11-01t00:00:00.000z,1981-04-15t00:00:00.000z,12/21/18,04/15/81,0,1,1992-10-01t00:00:00.000z,1999-10-01t00:00:00.000z,7.0,15%,05%,2023-09-19t20:31:29.315z
18,60015,amador county *,amador,ca,1977-06-07t00:00:00.000z,1984-09-24t00:00:00.000z,01/20/16,09/24/84,0,1,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:31:29.389z


In [16]:
len(isolated_flood_insurance_counties)

57

In [53]:
# Identify counties with no participation
isolated_flood_insurance_counties.loc[isolated_flood_insurance_counties.participatinginnfip == 0]

,communityidnumber,communityname,county,state,initialfloodhazardboundarymap,initialfloodinsuranceratemap,currentlyeffectivemapdate,regularemergencyprogramdate,tribal,participatinginnfip,originalentrydate,classratingeffectivedate,classrating,sfhadiscount,nonsfhadiscount,lastrefresh
532,60634,mariposa county *,mariposa,ca,1977-11-29t00:00:00.000z,1990-09-05t00:00:00.000z,09/25/09,11/29/78,0,0,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:33:43.193z


## Isolate relevant columns and adjust the entries within for increased transparency
* remove astrisks
* rename communityname column to county

In [51]:
flood_insurance_counties = isolated_flood_insurance_counties[['communityname', 'participatinginnfip']].copy()
flood_insurance_counties['communityname'] = flood_insurance_counties['communityname'].str.replace(' county *', '')
flood_insurance_counties['communityname'] = flood_insurance_counties['communityname'].str.replace(' county*', '')
flood_insurance_counties = flood_insurance_counties.rename(columns={'communityname':'county', 'participatinginnfip':'nfip_participation'})

flood_insurance_counties.head()

,county,nfip_participation
0,sacramento,1
4,monterey,1
6,alpine,1
7,alameda,1
18,amador,1


## Look at the original dataset to identify how many tribal participation entries there are

In [27]:
tribe_checking = community_flood_insurance_data[community_flood_insurance_data['tribal'] > 0]
tribe_checking

,communityidnumber,communityname,county,state,initialfloodhazardboundarymap,initialfloodinsuranceratemap,currentlyeffectivemapdate,regularemergencyprogramdate,tribal,participatinginnfip,originalentrydate,classratingeffectivedate,classrating,sfhadiscount,nonsfhadiscount,lastrefresh
310,60397,big valley band of pomo indians,lake,ca,NaN,2005-09-30t00:00:00.000z,09/30/05,06/14/10,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:31:33.111z
403,60743,fort mojave indian tribe,san bernardino,ca,NaN,1996-03-18t00:00:00.000z,08/28/08,03/18/96,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:31:34.231z
426,60763,agua caliente band of cahuilla indians tribe,riverside,ca,1974-06-21t00:00:00.000z,1983-03-02t00:00:00.000z,04/19/17,06/21/96,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-19t20:31:34.551z


## As there are only three separate counties with tribal nfip participation, we can manually add the flag for participation

In [50]:
community_flood_metric = flood_insurance_counties
# Add new column with default value
community_flood_metric['at_least_one_tribe_enrolled_within_county'] = 0

# List of specified counties to set to 1
specified_counties = ['lake', 'san bernardino', 'riverside']

# Update the 'specified_county' column to 1 for specified counties
community_flood_metric.loc[community_flood_metric['county'].isin(specified_counties), 'at_least_one_tribe_enrolled_within_county'] = 1
community_flood_metric

,county,nfip_participation,at_least_one_tribe_enrolled_within_county
0,sacramento,1,0
4,monterey,1,0
6,alpine,1,0
7,alameda,1,0
18,amador,1,0
20,butte,1,0
24,colusa,1,0
27,contra costa,1,0
36,el dorado,1,0
50,glenn,1,0


In [52]:
# save as a csv for upload to s3 bucket
community_flood_metric.to_csv('governance_community_flood_participation_metric.csv')

## Fema Policy metric

In [54]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/governance/personal_preparedness/fema/fema_fima_nfip_policies/fema_flood_redacted_policies.csv'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

KeyboardInterrupt: 

In [15]:
fema_flood_policy_data = pd.read_csv('fema_flood_redacted_policies.csv')
print(len(fema_flood_policy_data))
# os.remove('fema_flood_redacted_policies.csv')
pd.set_option('display.max_columns', None)
fema_flood_policy_data.tail(5)

/var/folders/6g/b2z199kx72x11z3xdvsz3ytr0000gn/T/ipykernel_7959/1564667749.py:1: DtypeWarning: Columns (4,11,39,40,50,58,69) have mixed types. Specify dtype option on import or set low_memory=False.
  fema_flood_policy_data = pd.read_csv('fema_flood_redacted_policies.csv')


4204541


,Unnamed: 0,agricultureStructureIndicator,baseFloodElevation,basementEnclosureCrawlspaceType,cancellationDateOfFloodPolicy,condominiumCoverageTypeCode,construction,crsClassCode,buildingDeductibleCode,contentsDeductibleCode,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,federalPolicyFee,ratedFloodZone,hfiaaSurcharge,houseOfWorshipIndicator,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,nonProfitIndicator,numberOfFloorsInInsuredBuilding,obstructionType,occupancyType,originalConstructionDate,originalNBDate,policyCost,policyCount,policyEffectiveDate,policyTerminationDate,policyTermIndicator,postFIRMConstructionIndicator,primaryResidenceIndicator,rateMethod,regularEmergencyProgramIndicator,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,totalInsurancePremiumOfThePolicy,cancellationVoidanceReasonCode,subsidizedRateType,iccPremium,reserveFundAssessment,communityProbationSurcharge,premiumPaymentIndicator,buildingReplacementCost,basicBuildingRate,additionalBuildingRate,basicContentsRate,AdditionalContentsRate,enclosureTypeCode,buildingDescriptionCode,insuranceToValueCode,postFirmVzoneIndicator,floodproofedIndicator,waitingPeriodType,rolloverTransferCode,endorsementEffectiveDate,propertyPurchaseDate,rentalPropertyIndicator,tenantIndicator,stateOwnedIndicator,disasterAssistanceCoverageRequiredCode,mandatoryPurchaseFlag,grandfatheringTypeCode,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,programTypeIndicator,mapPanelNumber,mapPanelSuffix,floodZoneCurrent,femaRegion,propertyState,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
4204536,152807,0,NaN,NaN,NaN,N,0,4.0,1,1,0,NaN,NaN,20,X,0,0,3.0,NaN,NaN,0,1.0,NaN,1.0,1964-07-01T00:00:00.000Z,2004-10-19T00:00:00.000Z,365,1,2011-10-19T00:00:00.000Z,2012-10-19T00:00:00.000Z,1,0,1,7,R,0,250000,100000.0,345,NaN,NaN,5.0,0,0.0,1.0,250000.0,0.00,0.00,0.00,0.00,NaN,1.0,NaN,0,0,NaN,R,2011-10-19T00:00:00.000Z,NaN,0,0,0,0.0,0,1.0,60262.0,NaN,SACRAMENTO COUNTY *,0,205.0,E,NaN,9.0,CA,Currently Unavailable,95670.0,6067.0,6.067009e+09,6.067009e+10,38.6,-121.3,6f40875d-8564-47e3-86a3-6b77160116c3
4204537,152808,0,NaN,NaN,NaN,N,0,4.0,1,1,0,NaN,NaN,20,X,0,0,3.0,NaN,NaN,0,1.0,NaN,1.0,1964-07-01T00:00:00.000Z,2004-10-19T00:00:00.000Z,376,1,2012-10-19T00:00:00.000Z,2013-10-19T00:00:00.000Z,1,0,1,7,R,0,250000,100000.0,356,NaN,NaN,5.0,0,0.0,1.0,250000.0,0.00,0.00,0.00,0.00,NaN,1.0,NaN,0,0,NaN,R,2012-10-19T00:00:00.000Z,NaN,0,0,0,0.0,0,1.0,60262.0,NaN,SACRAMENTO COUNTY *,0,205.0,E,NaN,9.0,CA,Currently Unavailable,95670.0,6067.0,6.067009e+09,6.067009e+10,38.6,-121.3,dc64d7c7-8358-4362-8a91-e67687412c7c
4204538,152984,0,NaN,NaN,NaN,N,0,NaN,2,NaN,0,NaN,NaN,30,AO,0,0,NaN,NaN,NaN,0,1.0,NaN,4.0,1990-07-01T00:00:00.000Z,2004-10-22T00:00:00.000Z,1572,1,2006-10-22T00:00:00.000Z,2007-10-22T00:00:00.000Z,1,1,0,1,R,0,275000,0.0,1542,NaN,NaN,6.0,0,0.0,NaN,275000.0,0.84,0.30,1.63,0.25,NaN,NaN,NaN,0,0,NaN,R,2006-10-22T00:00:00.000Z,NaN,0,0,0,0.0,0,NaN,60245.0,NaN,RIVERSIDE COUNTY *,0,3335.0,D,NaN,9.0,CA,Currently Unavailable,92240.0,6065.0,6.065045e+09,6.065045e+10,33.9,-116.5,dc524883-13fa-4d11-a5b5-0a354813aff9
4204539,152985,0,NaN,NaN,NaN,N,0,NaN,2,NaN,0,NaN,NaN,30,AO,0,0,NaN,NaN,NaN,0,1.0,NaN,4.0,1990-07-01T00:00:00.000Z,2004-10-22T00:00:00.000Z,1721,1,2007-10-22T00:00:00.000Z,2008-10-22T00:00:00.000Z,1,1,0,1,R,0,275000,0.0,1691,NaN,NaN,6.0,0,0.0,NaN,275000.0,0.92,0.33,1.80,0.28,NaN,NaN,NaN,0,0,NaN,R,2007-10-22T00:00:00.000Z,NaN,0,0,0,0.0,0,NaN,60245.0,NaN,RIVERSIDE COUNTY *,0,3335.0,D,NaN,9.0,CA,Currently Unavailable,92240.0,6065.0,6.065045e+09,6.065045e+10,33.9,-116.5,3d12e55b-41fe-44c8-9a0f-c847dedcd420
4204540,152986,0,NaN,NaN,NaN,N,0,NaN,2,NaN,0,NaN,NaN,35,AO,0,0,NaN,NaN,NaN,0,1.0,NaN,4.0,1990-07-01T00:00:00.000Z,2004-10-22T00:00:00.000Z,1888,1,2008-10-22T00:00:00.000Z,2009-10-22T00:00:00.000Z,1,1,0,1,R,0,275000,0.0,1853,NaN,NaN,6.0,0,0.0,NaN,275000.0,1.01,0.36,1.97,0.31

In [34]:
len(fema_flood_policy_data.loc[fema_flood_policy_data['censusTract'] == 6.065045e+09])

229

In [16]:
fema_flood_policy_data.columns

Index(['Unnamed: 0', 'agricultureStructureIndicator', 'baseFloodElevation',
       'basementEnclosureCrawlspaceType', 'cancellationDateOfFloodPolicy',
       'condominiumCoverageTypeCode', 'construction', 'crsClassCode',
       'buildingDeductibleCode', 'contentsDeductibleCode',
       'elevatedBuildingIndicator', 'elevationCertificateIndicator',
       'elevationDifference', 'federalPolicyFee', 'ratedFloodZone',
       'hfiaaSurcharge', 'houseOfWorshipIndicator', 'locationOfContents',
       'lowestAdjacentGrade', 'lowestFloorElevation', 'nonProfitIndicator',
       'numberOfFloorsInInsuredBuilding', 'obstructionType', 'occupancyType',
       'originalConstructionDate', 'originalNBDate', 'policyCost',
       'policyCount', 'policyEffectiveDate', 'policyTerminationDate',
       'policyTermIndicator', 'postFIRMConstructionIndicator',
       'primaryResidenceIndicator', 'rateMethod',
       'regularEmergencyProgramIndicator', 'smallBusinessIndicatorBuilding',
       'totalBuildingInsuran

In [17]:
# Display the number of non-NaN values in each column
non_nan_counts = fema_flood_policy_data.count()

# Set display option to show all rows
pd.set_option('display.max_rows', None)

# Display the counts
print(non_nan_counts)

Unnamed: 0                                4204541
agricultureStructureIndicator             4204541
baseFloodElevation                         631482
basementEnclosureCrawlspaceType           1506893
cancellationDateOfFloodPolicy              171330
condominiumCoverageTypeCode               4202840
construction                              4204541
crsClassCode                              2675842
buildingDeductibleCode                    4160744
contentsDeductibleCode                    3225311
elevatedBuildingIndicator                 4204541
elevationCertificateIndicator              848567
elevationDifference                        649410
federalPolicyFee                          4204541
ratedFloodZone                            4197244
hfiaaSurcharge                            4204541
houseOfWorshipIndicator                   4204541
locationOfContents                        2392691
lowestAdjacentGrade                        550886
lowestFloorElevation                       642944


In [20]:
# Convert the 'policyTerminationDate' column to datetime
fema_flood_policy_data['policyTerminationDate'] = pd.to_datetime(fema_flood_policy_data['policyTerminationDate'], errors='coerce')

# Create a mask for dates after 2023
mask = (fema_flood_policy_data['policyTerminationDate'].dt.year > 2023) & (fema_flood_policy_data['policyTerminationDate'].dt.year != 2203)

# Apply the mask to filter the data
current_flood_policy = fema_flood_policy_data[mask]

print(len(current_flood_policy))
current_flood_policy.head(5)

126882


,Unnamed: 0,agricultureStructureIndicator,baseFloodElevation,basementEnclosureCrawlspaceType,cancellationDateOfFloodPolicy,condominiumCoverageTypeCode,construction,crsClassCode,buildingDeductibleCode,contentsDeductibleCode,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,federalPolicyFee,ratedFloodZone,hfiaaSurcharge,houseOfWorshipIndicator,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,nonProfitIndicator,numberOfFloorsInInsuredBuilding,obstructionType,occupancyType,originalConstructionDate,originalNBDate,policyCost,policyCount,policyEffectiveDate,policyTerminationDate,policyTermIndicator,postFIRMConstructionIndicator,primaryResidenceIndicator,rateMethod,regularEmergencyProgramIndicator,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,totalInsurancePremiumOfThePolicy,cancellationVoidanceReasonCode,subsidizedRateType,iccPremium,reserveFundAssessment,communityProbationSurcharge,premiumPaymentIndicator,buildingReplacementCost,basicBuildingRate,additionalBuildingRate,basicContentsRate,AdditionalContentsRate,enclosureTypeCode,buildingDescriptionCode,insuranceToValueCode,postFirmVzoneIndicator,floodproofedIndicator,waitingPeriodType,rolloverTransferCode,endorsementEffectiveDate,propertyPurchaseDate,rentalPropertyIndicator,tenantIndicator,stateOwnedIndicator,disasterAssistanceCoverageRequiredCode,mandatoryPurchaseFlag,grandfatheringTypeCode,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,programTypeIndicator,mapPanelNumber,mapPanelSuffix,floodZoneCurrent,femaRegion,propertyState,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
6029,22936,0,1.0,0.0,NaN,N,0,NaN,2,NaN,1,3,2.0,47,AO,25,0,NaN,0.0,2.5,0,1.0,10.0,11.0,1960-01-01T00:00:00.000Z,2001-07-20T00:00:00.000Z,487,1,2023-07-20T00:00:00.000Z,2024-07-20 00:00:00+00:00,1,0,1,RatingEngine,R,0,145000,0.0,352,NaN,NaN,8.0,63,NaN,2.0,60000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,N,R,NaN,NaN,0,0,0,0.0,0,1.0,60017.0,60017.0,BUTTE COUNTY *,0,505.0,E,AOB,9.0,CA,Currently Unavailable,95928.0,6007.0,6.007001e+09,6.007001e+10,39.7,-121.8,96fdb2fd-1c65-45e7-bbac-e5587368662f
12434,113407,0,NaN,2.0,NaN,N,0,8.0,F,F,0,NaN,NaN,47,A99,250,0,2.0,NaN,NaN,0,1.0,NaN,13.0,2001-01-19T00:00:00.000Z,2008-07-30T00:00:00.000Z,1050,1,2023-07-30T00:00:00.000Z,2024-07-30 00:00:00+00:00,1,1,0,RatingEngine,R,0,500000,8000.0,638,NaN,NaN,75.0,115,NaN,2.0,250000.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0,0,N,N,2023-08-21T00:00:00.000Z,2008-07-30T00:00:00.000Z,0,0,0,NaN,0,1.0,60294.0,60294.0,"OCEANSIDE, CITY OF",0,753.0,J,X,9.0,CA,Currently Unavailable,92058.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,6fe5e910-8e23-4204-b9ea-b8994b840e79
21288,9534,0,NaN,0.0,NaN,N,0,7.0,2,1,0,NaN,NaN,47,X,25,0,3.0,NaN,NaN,0,1.0,NaN,11.0,1923-07-01T00:00:00.000Z,2023-01-14T00:00:00.000Z,729,1,2023-01-14T00:00:00.000Z,2024-01-14 00:00:00+00:00,1,0,1,RatingEngine,R,0,200000,80000.0,557,NaN,NaN,13.0,100,NaN,2.0,175000.0,NaN,NaN,NaN,NaN,N,1.0,NaN,0,0,NaN,Z,NaN,NaN,0,0,0,0.0,0,1.0,60137.0,60137.0,"LOS ANGELES, CITY OF",0,1781,G,X,9.0,CA,Currently Unavailable,90047.0,6037.0,6.037237e+09,6.037237e+10,34.0,-118.3,89c083c9-60a8-4933-929e-4b29d9409b17
34260,28689,0,436.0,NaN,NaN,N,0,9.0,F,NaN,0,3,2.0,47,AO,25,0,NaN,433.5,437.5,0,1.0,NaN,11.0,1999-01-01T00:00:00.000Z,2010-01-13T00:00:00.000Z,798,1,2023-01-05T00:00:00.000Z,2024-01-05 00:00:00+00:00,1,1,1,RatingEngine,R,0,235000,0.0,615,NaN,NaN,27.0,111,NaN,1.0,176300.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0,0,N,R,2023-01-05T00:00:00.000Z,1999-01-01T00:00:00.000Z,0,0,0,NaN,0,3.0,60075.0,60076.0,KERN COUNTY*,0,1100.0,C,AOB,9.0,CA,Currently Unavailable,93203.0,6029.0,6.029006e+09,6.029006e+10,35.2,-118.8,408ecf60-b0c7-42e3-9c04-ae874f29854a
34855,37102,0,NaN,NaN,NaN,N,0,7.0,2,NaN,0,NaN,NaN,47,AO,250,0,NaN,NaN,NaN,0,1.0,NaN,11.0,1967-01-01T00:00:00.000Z,2010-05-18T00:00:00.000Z,1140,1,2023-07-26T00:00:00.000Z,2024-07-26 00:00:00+00:00,1,0,0,RatingEngine,R,0,250000,0.0,714,NaN,NaN

In [21]:
# Convert the 'policyTerminationDate' column to datetime
filtered_data['policyTerminationDate'] = pd.to_datetime(filtered_data['policyTerminationDate'], errors='coerce')

# Extract the year and get unique years
unique_years = filtered_data['policyTerminationDate'].dt.year.unique()

# Display the unique years
print(sorted(unique_years))

NameError: name 'filtered_data' is not defined

In [24]:
len(current_flood_policy['censusTract'].unique())

6152

In [25]:
current_flood_policy['censusTract'].unique()

array([6.0070011e+09, 6.0730186e+09, 6.0372373e+09, ..., 6.0590639e+09,
       6.0374627e+09, 6.0530121e+09])

In [27]:
current_flood_policy.loc[current_flood_policy['nfipCommunityName'] == "BUTTE COUNTY *"]['censusTract'].unique()

array([6.0070011e+09, 6.0070015e+09, 6.0070013e+09, 6.0070014e+09,
       6.0070034e+09, 6.0070036e+09, 6.0070016e+09, 6.1110009e+09,
       6.0070035e+09, 6.0070004e+09, 6.0070030e+09, 6.0070012e+09,
       6.0070005e+09, 6.0070033e+09, 6.0070002e+09, 6.0070032e+09,
       6.0070009e+09, 6.0070022e+09, 6.0070027e+09, 6.0070008e+09,
       6.0070003e+09,           nan, 6.0070005e+09, 6.0070037e+09,
       6.0070031e+09, 6.0070035e+09, 6.1150410e+09, 6.0070023e+09,
       6.0070009e+09, 6.0070026e+09, 6.0070029e+09, 6.0070009e+09,
       6.0070024e+09])